In [33]:
import os
import zipfile
import cdsapi
import xarray as xr

import numpy as np

from scipy.interpolate import RegularGridInterpolator

R = 8.314 #J/(mol*K)
M_air = 0.02896 #kg/mol

MAPEH1 = (22.600338,114.545072)
MAPEH2 = (22.608738,114.548871)
MAPEH3 = (22.614854,114.540537)

MAPzEH1 = -21.79
MAPzEH2 = 4.44
MAPzEH3 = 10.07

In [34]:
'''
指定数据的时间
'''
dayRequest = 15
monthRequest = 10
yearRequest = 2024

'''
指定数据的空间范围。根据测算的推荐值为N0-N40, E100-E130
'''
northBoundary = 40
southBoundary = 0
westBoundary  = 100
eastBoundary  = 130
areaRequest = [northBoundary,westBoundary,southBoundary,eastBoundary]

'''
给定数据的存储位置, zip文件保存在当前目录下/data/子目录，以时间命名。解压缩文件放在时间命名的文件夹下。
'''
dateStr = f'{yearRequest}{monthRequest}{dayRequest}'
currentPath = os.getcwd()
zipPath  = currentPath + f'/data/{dateStr}.zip'
dataPath = currentPath + f'/data/{dateStr}/'
geopDataPath = dataPath + 'geopotential_stream-oper_daily-mean.nc' # geop文件默认名称
tempDataPath = dataPath + 'temperature_0_daily-mean.nc'            # temp文件默认名称

'''
尝试打开指定当前路径下/data/{dateStr}/路径下的重力势数据和温度数据，否则下载新的文件
'''
try: 
    geopDataSet = xr.open_dataset(geopDataPath,engine='netcdf4')
    tempDataSet = xr.open_dataset(tempDataPath,engine='netcdf4')
except FileNotFoundError:
    dataset = "derived-era5-pressure-levels-daily-statistics"
    request = {
    "product_type": "reanalysis",
    "variable": [
        "geopotential",
        "temperature"
    ],
    "year": f"{yearRequest}",
    "month": [f"{monthRequest}"],
    "day": [f"{dayRequest}"],
    "pressure_level": [
        "1", "2", "3",
        "5", "7", "10",
        "20", "30", "50",
        "70", "100", "125",
        "150", "175", "200",
        "225", "250", "300",
        "350", "400", "450",
        "500", "550", "600",
        "650", "700", "750",
        "775", "800", "825",
        "850", "875", "900",
        "925", "950", "975",
        "1000"
    ],
    "daily_statistic": "daily_mean",
    "time_zone": "utc+08:00",
    "frequency": "6_hourly",
    "area": areaRequest
}
    client = cdsapi.Client()
    client.retrieve(dataset, request).download(zipPath)

    with zipfile.ZipFile(zipPath,'r') as z:
        fileList = z.namelist()
        print(f'{fileList}')
        os.mkdir(currentPath+f'/data/{dateStr}')
        z.extractall(currentPath+f'/data/{dateStr}/')
        geopDataSet = xr.open_dataset(currentPath+f'/data/{dateStr}/{fileList[0]}',engine='netcdf4')
        tempDataSet = xr.open_dataset(currentPath+f'/data/{dateStr}/{fileList[1]}',engine='netcdf4')

'''
如果是新下载的文件或者没有删除干净, 重新删除zip文件
'''
try:
    os.remove(zipPath)
except FileNotFoundError:pass

temperature = tempDataSet['t'].squeeze()
altitude = geopDataSet['z'].squeeze()/9.8066
pressure = temperature['pressure_level']*1e2 # pa
longitude = temperature['longitude']
latitude  = temperature['latitude']

density = pressure * M_air /(R * temperature)

tempInterp = RegularGridInterpolator((pressure,latitude,longitude),values=temperature)
densInterp = RegularGridInterpolator((pressure,latitude,longitude),values=density)
altiInterp = RegularGridInterpolator((pressure,latitude,longitude),values=altitude)

In [35]:
tempInterp((1.5e3,*MAPEH1))

array(226.81513979)